## AGAME Workflow. Site selection

In [1]:
import os
import pandas as pd
import numpy as np
import zipfile
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from icoscp.station import station
from icoscp.dobj import Dobj
import geopandas as gpd
from shapely.geometry import Point
import folium
import requests

# from icoscp_core.icos import auth
# auth.init_config_file()

In [5]:
def read_flux_df(file_path):
    print('\nReading:', file_path)

    df = pd.read_csv(file_path, na_values=-9999)
    original_num_columns = df.shape[1]
    cols_to_drop = df.columns[df.isna().all()].tolist()
    num_cols_dropped = len(cols_to_drop)

    print(f"Original number of columns: {original_num_columns}")
    print(f"Number of columns dropped: {num_cols_dropped}")
    print(f"Columns dropped: {cols_to_drop}\n")

    df_cleaned = df.drop(columns=cols_to_drop)

    return df_cleaned

def filter_era_columns(sites_table, df, list_col, nan_files):
    counts = Counter(list_col)
    list_col_in_all = set([item for item in list_col if counts[item] == (len(sites_table)-len(nan_files))])
    print(f"Number of columns available in all files: {len(list_col_in_all)}\n")

    filtered_list_col_in_all = sorted([item for item in list_col_in_all if not (item.startswith('NEE') or 
                                                                                item.startswith('GPP') or 
                                                                                item.startswith('TIMESTAMP') or
                                                                                item.endswith('_QC') or 
                                                                                item.endswith('_METHOD') or 
                                                                                item.startswith('RECO'))])

    combined_var = sorted([item for item in list_col_in_all  if item.endswith('_F')])
    era_var = sorted([item for item in list_col_in_all  if item.endswith('_ERA')])
    mds_var = sorted([item for item in list_col_in_all  if item.endswith('_MDS')])

    era_var.insert(0, 'GPP_DT_VUT_REF')
    df = df[era_var]

    correlation_matrix = df.corr()

    plt.figure(figsize=(10, 12))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title('Correlation Matrix')
    plt.show()

    return df

def filter_flux_columns(data_directory, sites_table):

    list_col = []
    nan_files = []

    for index, row in sites_table.iterrows():
        print(f"\nSite ID: {row['sites_ids']}")

        # Unzip files if needed
        # file_path = os.path.join(data_directory, row['File name'])
        # with zipfile.ZipFile(file_path, 'r') as zip_ref:
        #     zip_ref.extractall(os.path.join(data_directory, row['sites_folder'])) 
        #     print(f"Unzipped: {row['sites_folder']}")
        try:
            flux_file_path = os.path.join(data_directory,row['sites_folder'],f"ICOSETC_{row['sites_ids']}_FLUXNET_DD_L2.csv" )
            flux_df = read_flux_df(flux_file_path)
            list_col.extend(flux_df.columns.values.tolist())

        except FileNotFoundError:
            print(f"File not found: {flux_file_path}")
            nan_files.append({flux_file_path}) 

    counts = Counter(list_col)
    list_col_in_all = set([item for item in list_col if counts[item] == (len(sites_table)-len(nan_files))])
    print(f"Number of columns available in all files: {len(list_col_in_all)}")

    filtered_flux_columns = sorted([item for item in list_col_in_all if not (item.startswith('NEE') or 
                                                                                item.startswith('GPP') or 
                                                                                item.startswith('TIMESTAMP') or
                                                                                item.endswith('_QC') or 
                                                                                item.endswith('_METHOD') or 
                                                                                item.startswith('RECO'))])
    
    print(f"Number of columns available in all files without carbon data: {len(filtered_flux_columns)}\n")

    insitu_cols = ['GPP_DT_VUT_REF'] + filtered_flux_columns #'GPP_DT_VUT_USTAR50'

    return insitu_cols

def get_eu_sites_noname(df):
    # Fetch country data from the REST Countries API
    response = requests.get('https://restcountries.com/v3.1/all')
    countries = response.json()

    # Extract country codes and their regions
    european_country_codes = [country['cca2'] for country in countries if 'region' in country and country['region'] == 'Europe']
    df['IsInEurope'] = df['country'].isin(european_country_codes)
    df = df[df['IsInEurope'] == True]
    df = df.drop(columns=['IsInEurope'])
    df
    return df

def get_eu_sites(df):
    # Fetch country data from the REST Countries API
    response = requests.get('https://restcountries.com/v3.1/all')
    countries = response.json()

    # Create a dictionary mapping country codes to their common names
    country_code_to_name = {country['cca2']: country['name']['common'] for country in countries if 'region' in country and country['region'] == 'Europe'}
    
    # Filter the DataFrame to include only European countries
    df['IsInEurope'] = df['country'].isin(country_code_to_name)
    df = df[df['IsInEurope'] == True]
    
    # Add a new column for the common name of the country
    df['CountryName'] = df['country'].map(country_code_to_name)
    
    # Drop the 'IsInEurope' column
    df = df.drop(columns=['IsInEurope'])
    
    return df


def map_sites(df):
    # Convert lat/lon to geometry
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Create a Folium map centered around the average location
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=5)

    # Add points to the map
    for idx, row in gdf.iterrows():
        folium.Marker([row['lat'], row['lon']], popup=row['siteType']).add_to(m)

    # Display the map
    return m

def map_sites_with_colors(df):
    
    # Convert lat/lon to geometry
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)

    # Define a color mapping based on the name
    color_mapping = {
        'Cropland': 'yellow',
        'cropland': 'yellow',
        'Forest': 'green',
        'forest': 'green',
        'Wetland': 'blue',
        'wetland': 'blue',
        'Grassland': 'magenta',
        'grassland': 'magenta',
        'Heathland': 'red',
        'Urban': 'orange',
        'urban': 'orange'
    }

    # Create a Folium map centered around the average location
    m = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=3)

    # Add points to the map with different colors
    for idx, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4,
            popup=row['siteType'],
            color=color_mapping.get(row['siteType'], 'black'),  # Default to black if name not in mapping
            fill=True,
            fill_color=color_mapping.get(row['siteType'], 'black')
        ).add_to(m)

    # Display the map
    return m

def filter_long_ts(df, years):
    seconds_in_year = 365.25 * 24 * 60 * 60

    df['years_HH'] = df['end_HH'] - df['start_HH'] 
    df['years_HH']=df['years_HH'].dt.total_seconds() / seconds_in_year

    df = df[df['years_HH'] >= years]

    return df

def get_sites(df, years_ts, output_directory):

    df.to_csv(os.path.join(output_directory,'sites_table_complete.csv'), index=False)

    df = get_eu_sites(df)
    df = filter_long_ts(df, years_ts)
    df = df[df['ecosystemType'] != '']
    df['siteType'] = df['siteType'].str.strip()
    df['ecosystemType'] = df['ecosystemType'].str.strip()
    df['siteType'] = df['siteType'].str.lower()
    df['ecosystemType'] = df['ecosystemType'].str.lower()

    df = df[(df['siteType'] == 'forest') | (df['siteType'] == 'grassland')]

    df = df.dropna(subset=['FETCH_70'])

    df.to_csv(os.path.join(output_directory,'sites_table_filtered_4y.csv'), index=False) 
    print(df.shape)
    return df

def read_icos_attributes(index, sites_table, id, output_directory,axs):
    myStation = station.get(id)
    metadata = myStation.info()
    sites_table.loc[index, 'name'] = metadata['name']
    sites_table.loc[index, 'uri'] = metadata['uri'][0]
    sites_table.loc[index, 'lat'] = metadata['lat']
    sites_table.loc[index, 'lon'] = metadata['lon']
    sites_table.loc[index, 'eas'] = metadata['eas'] 
    sites_table.loc[index, 'country'] = metadata['country']
    sites_table.loc[index, 'siteType'] = metadata['siteType']

    nan_ids = []
    try:
        selected_df = myStation.data(2).loc[myStation.data(2)['specLabel'] == "ETC L2 Fluxnet (half-hourly)"]
        pid = selected_df['dobj'].values.tolist()
        dobj = Dobj(pid[0])

        start, end = dobj.data['TIMESTAMP'].iloc[[0, -1]]
        sites_table.loc[index, 'start_HH'] = start
        sites_table.loc[index, 'end_HH'] = end

        # ax = dobj.data.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True)
        # ax.set_title(f'GPP_DT_VUT_REF Over Time at {id} Station')

        dobj.data.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[0])
        axs[0].set_title(f'GPP_DT_VUT_REF Over Time at {id} Station HH')

        # plt.savefig(os.path.join(output_directory, f'GPP_{id}.png'))
        # plt.close()

        sites_table.loc[index, 'ecosystemType'] = dobj.meta['specificInfo']['acquisition']['station']['specificInfo']['ecosystemType']['label']
        dobj = dobj.data
        
    except FileNotFoundError:
        print(f"Flux data not found: {id}")
        nan_ids.append({id})
        dobj = pd.DataFrame() 
    except IndexError:
        print(f"Index out of range for station {id}")
        nan_ids.append(id)
        dobj = pd.DataFrame() 
    except KeyError as e:
        print(f"KeyError: {e}. The key 'GPP_DT_VUT_REF' does not exist for station {id}.")
        nan_ids.append(id)
        dobj = pd.DataFrame() 

    return sites_table, nan_ids, axs, dobj


def process_insitu_data(data_directory, sites_folder, id, insitu_cols, sites_table, index, output_directory, axs):

    nan_files = []
    try:
        flux_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXNET_DD_L2.csv" )
        insitu = pd.read_csv(flux_file_path, index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])
        insitu = insitu[insitu_cols]

        insitu['TIMESTAMP'] = pd.to_datetime(insitu.index)

        start, end = insitu['TIMESTAMP'].iloc[[0, -1]]
        sites_table.loc[index, 'start_DD'] = start
        sites_table.loc[index, 'end_DD'] = end

        insitu.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[1])
        axs[1].set_title(f'GPP_DT_VUT_REF Over Time at {id} Station (Daily Data)')

        fetch_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXES_L2.csv" )
        fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)

        sites_table.loc[index, 'FETCH_MAX'] = fetch.describe()['FETCH_MAX'].loc['50%']
        sites_table.loc[index, 'FETCH_70'] = fetch.describe()['FETCH_70'].loc['50%']
        sites_table.loc[index, 'FETCH_90'] = fetch.describe()['FETCH_90'].loc['50%']

        iheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_VARINFO_FLUXNET_DD_L2.csv" )
        iheight = pd.read_csv(iheight_file_path)
        filtered_df = iheight[iheight['DATAVALUE'] == 'GPP_DT_VUT_REF']
        index_gpp = filtered_df.index.values.tolist()[0] 
        iheight = iheight[index_gpp:index_gpp+5]
        iheight_value = iheight[iheight['VARIABLE'] == 'VAR_INFO_HEIGHT']
        sites_table.loc[index, 'instrument_height'] = float(iheight_value['DATAVALUE'].values.tolist()[0])

        cheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_ANCILLARY_L2.csv" )
        cheight = pd.read_csv(cheight_file_path)
        cheight = cheight[cheight['VARIABLE_GROUP'] == 'GRP_HEIGHTC']
        cheight = cheight[cheight['VARIABLE'] == 'HEIGHTC']
        cheight['DATAVALUE'] = cheight['DATAVALUE'].astype(float)
        sites_table.loc[index, 'canopy_height'] = cheight.describe()['DATAVALUE'].loc['75%']
        sites_table['buffer'] = 100 * (sites_table['instrument_height']-sites_table['canopy_height']*2/3) #Fetch to height ratio https://www.mdpi.com/2073-4433/10/6/299
                                                                                                        #https://nicholas.duke.edu/people/faculty/katul/Matlab_footprint.html 
        sites_table['buffer_desired'] = sites_table['buffer'] / 3
        sites_table['buffer_difference'] = sites_table['buffer_desired'] - sites_table['FETCH_90']

    except FileNotFoundError:
        print(f"File not found: {flux_file_path}")
        nan_files.append({flux_file_path}) 
        insitu = pd.DataFrame() 

    except KeyError as e:
        print(f"KeyError: {e}. The key 'FETCH' does not exist for station {id}.")

    return sites_table, nan_files, axs, insitu


def plot_gpp(dobj, insitu, axs, output_directory):
    nan_plots = []
    try:
        # Plot 3: Combined Plot
        if 'GPP_DT_VUT_REF' in dobj.columns and 'GPP_DT_VUT_REF' in insitu.columns:
            # Plot 3
            dobj.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[2], label='Dobj Data')
            insitu.plot(x='TIMESTAMP', y='GPP_DT_VUT_REF', grid=True, ax=axs[2], label='Daily Data', linestyle='--')
            axs[2].set_title(f'Combined GPP_DT_VUT_REF Over Time at {id} Station')
            axs[2].legend()

        plt.tight_layout()
        plt.savefig(os.path.join(output_directory, f'Combined_GPP_{id}.png'))
        plt.close()

    except KeyError as e:
        print(f"KeyError: {e}. The key 'GPP_DT_VUT_REF' does not exist in one of the dataframes for station {id}.")
        nan_plots.append(id)

    return axs, nan_plots

In [3]:
data_directory = r'D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC'
output_directory = r'D:\Proyectos2024\Agame\Output\sites_selection'

sites_table = pd.read_csv(os.path.join(data_directory,'!TOC.csv'))
sites_table['sites_ids'] = sites_table['File name'].apply(lambda x: x.split('_')[1])
sites_table['sites_folder'] = sites_table['File name'].apply(lambda x: x.replace('.zip', ''))
insitu_cols = filter_flux_columns(data_directory, sites_table)

sites_table['start_DD'] = pd.NaT
sites_table['end_DD'] = pd.NaT
sites_table['start_HH'] = pd.NaT
sites_table['end_HH'] = pd.NaT
sites_table['name'] = 'name'
sites_table['uri'] = 'uri'
sites_table['lat'] = np.nan
sites_table['lon'] = np.nan
sites_table['eas'] = np.nan
sites_table['country'] = ''
sites_table['siteType'] = ''
sites_table['ecosystemType'] = ''
sites_table['FETCH_MAX'] = np.nan
sites_table['FETCH_70'] = np.nan
sites_table['FETCH_90'] = np.nan
sites_table['canopy_height'] = np.nan
sites_table['instrument_height'] = ''

for index, row in sites_table.iterrows():
    
    id = row['sites_ids']
    sites_folder = row['sites_folder']
    print('Process for: ', id)

    fig, axs = plt.subplots(3, 1, figsize=(10, 15))

    sites_table, nan_ids, axs, dobj = read_icos_attributes(index, sites_table, id, output_directory, axs)
    sites_table, nan_files, axs, insitu = process_insitu_data(data_directory, sites_folder, id, insitu_cols, sites_table, index, output_directory, axs)
    axs, nan_plots = plot_gpp(dobj, insitu, axs, output_directory)


Site ID: FI-Sii

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_FI-Sii_ARCHIVE_L2\ICOSETC_FI-Sii_FLUXNET_DD_L2.csv
Original number of columns: 346
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: IT-SR2

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_IT-SR2_ARCHIVE_L2\ICOSETC_IT-SR2_FLUXNET_DD_L2.csv
Original number of columns: 346
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: SE-Htm

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_SE-Htm_ARCHIVE_L2\ICOSETC_SE-Htm_FLUXNET_DD_L2.csv
Original number of columns: 348
Number of columns dropped: 2
Columns dropped: ['RECO_SR', 'RECO_SR_N']


Site ID: GL-Dsk

Reading: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_GL-Dsk_ARCHIVE_L2\ICOSETC_GL-Dsk_FLUXNET_DD_L2.csv
Original number of columns: 344
Number of columns dropped: 14
Columns dropped: ['LE_F_MDS_QC', 'LE_CORR', 'LE_CORR_25', 'LE_CORR_75', 'LE_CORR_JOINTUNC', 'H

C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\820771650.py:266: DtypeWarning: Columns (174,176) have mixed types. Specify dtype option on import or set low_memory=False.
  fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)


Process for:  SE-Htm


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\820771650.py:266: DtypeWarning: Columns (182,184) have mixed types. Specify dtype option on import or set low_memory=False.
  fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)


Process for:  GL-Dsk
Process for:  GF-Guy
KeyError: 'FETCH_MAX'. The key 'FETCH' does not exist for station GF-Guy.
Process for:  FI-Kmp
Index out of range for station FI-Kmp
File not found: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_FI-Kmp_ARCHIVE_L2\ICOSETC_FI-Kmp_FLUXNET_DD_L2.csv
Process for:  CD-Ygb
Process for:  IT-BFt
KeyError: 'FETCH_MAX'. The key 'FETCH' does not exist for station IT-BFt.
Process for:  IT-Tor
Process for:  GR-HeM
Index out of range for station GR-HeM
File not found: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_GR-HeM_ARCHIVE_L2\ICOSETC_GR-HeM_FLUXNET_DD_L2.csv
Process for:  GR-HeK
Index out of range for station GR-HeK
File not found: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_GR-HeK_ARCHIVE_L2\ICOSETC_GR-HeK_FLUXNET_DD_L2.csv
Process for:  FR-CLt
Process for:  DE-BeR
Index out of range for station DE-BeR
File not found: D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC\ICOSETC_DE-BeR_ARCHIVE_L2\ICOSETC_DE-BeR_FLUXNET_DD_L2.csv
Pr

...

C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\820771650.py:266: DtypeWarning: Columns (202,204) have mixed types. Specify dtype option on import or set low_memory=False.
  fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)


Process for:  BE-Maa
Process for:  BE-Lon
Process for:  BE-Bra


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\820771650.py:266: DtypeWarning: Columns (182,184) have mixed types. Specify dtype option on import or set low_memory=False.
  fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)


Process for:  FR-LGt
KeyError: 'FETCH_MAX'. The key 'FETCH' does not exist for station FR-LGt.
Process for:  DK-Skj
Process for:  DK-Gds
Process for:  BE-Lcr
KeyError: 'FETCH_MAX'. The key 'FETCH' does not exist for station BE-Lcr.
(25, 29)


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\820771650.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CountryName'] = df['country'].map(country_code_to_name)


In [9]:
df = get_sites(sites_table, 3.9, output_directory)
map = map_sites_with_colors(df)
map

(18, 29)


C:\Users\fuentesm\AppData\Local\Temp\ipykernel_19564\223477878.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CountryName'] = df['country'].map(country_code_to_name)
